In [1]:
# 저번시간
# SQLite 를 이용해서 파이썬을 통해 코딩해봤음
# 커넥션 개체를 통해 데이터베이스에 접근한 뒤 커서를 부여 받아 작업을 시작함.
# 실제 작업을 시킬 때는 execute, executemany(iterable한 객체를 이용해서 반복적으로 수행할 때), executescript(여러 개 수행할 때)
# 데이터를 가져올 때는 fetchone() -> 한 개 값 리턴 해줌, fetchall() -> list에 담아서 리턴을 해줌
#! fetch를 한 번 하면 커서의 위치가 바뀜. 다른 작업을 해야하기 때문에. 따라서 두 번 fetch하면 두번째는 아무것도 없는 것을 확인 가능함

In [2]:
import sqlite3

In [3]:
# dump file. -> 나중에 한 번 해보기

In [ ]:
# 예제 - 프렌차이즈 전산실
# CITY, SUPPLIER, PART, SELLS(관계를 나타내는 TABLE)
# CITY: PK, NAME
# SUPLLIER: PK, NAME, FK(CITY.PK)
# PART: PK, NAME
# SELLS: FK(S.PK), FK(P.PK), PRICE

In [3]:
con = sqlite3.connect('example3.db')
cur = con.cursor()

In [4]:
cur.executescript('''
    CREATE TABLE city (
        pk INTEGER PRIMARY KEY,
        name TEXT
    );
    CREATE TABLE supplier (
        pk INTEGER PRIMARY KEY,
        name TEXT,
        fk INTEGER NOT NULL
    );
    CREATE TABLE part (
        pk INTEGER PRIMARY KEY,
        name TEXT
    );
    CREATE TABLE sells (
        fk1 INTEGER NOT NULL,
        fk2 INTEGER NOT NULL,
        price INTEGER NOT NULL
    );
''')

In [5]:
data = [('성북구',), ('중구',), ('강북구',), ('어쩌구',), ('저쩌구',)]
cur.executemany('INSERT INTO city(name) VALUES (?)', data)

In [6]:
cur.execute('SELECT * FROM city')
cur.fetchall()

[(1, '성북구'), (2, '중구'), (3, '강북구'), (4, '어쩌구'), (5, '저쩌구')]

In [7]:
data = [('메뉴1',), ('메뉴2',), ('메뉴3',), ('메뉴4',), ('메뉴5',)]
cur.executemany('INSERT INTO part(name) VALUES (?)', data)

In [8]:
cur.execute('SELECT * FROM part')
cur.fetchall()

[(1, '메뉴1'), (2, '메뉴2'), (3, '메뉴3'), (4, '메뉴4'), (5, '메뉴5')]

In [9]:
# FK 를 위해 어느 도시에 속했는지
cur.execute('SELECT * FROM city')
for row in cur.fetchall():
    if row[1] == '성북구':
        print(row[0])

1


In [10]:
cur.execute('SELECT * FROM city')
for row in cur.fetchall():
    if row[1] == '성북구':
        pk = row[0]
        cur.execute('INSERT INTO supplier (name, fk) VALUES(?, ?)',
                   ['안암 1호점', pk])
        break

In [11]:
cur.execute('SELECT * FROM supplier')
cur.fetchall()

[(1, '안암 1호점', 1)]

In [12]:
cur.execute('SELECT pk FROM city WHERE name LIKE :name', {'name':'%성북%'})
pk = cur.fetchone()[0]
cur.execute('INSERT INTO supplier (name, fk) VALUES(?, ?)',
           ['안암 2호점', pk])

In [13]:
cur.execute('SELECT * FROM supplier')
cur.fetchall()

[(1, '안암 1호점', 1), (2, '안암 2호점', 1)]

In [14]:
# 쿼리문 중첩 방법
cur.execute('''
    INSERT INTO supplier(name, fk)
    VALUES(?,(SELECT pk FROM city WHERE name LIKE ? LIMIT 0,1))
    ''', ['종암 1호점','%성북%'])

In [15]:
cur.execute('SELECT * FROM supplier')
cur.fetchall()

[(1, '안암 1호점', 1), (2, '안암 2호점', 1), (3, '종암 1호점', 1)]

In [25]:
con.commit()

In [17]:
data = ['%안암%1호%', '%메뉴1%', 4500]

cur.execute('''
    INSERT INTO sells(fk1, fk2, price)
    VALUES((SELECT pk FROM supplier WHERE name LIKE ? LIMIT 0,1),
           (SELECT pk FROM part WHERE name LIKE ? LIMIT 0,1),
           ?)
    ''', data)

In [18]:
data = [['%안암%1호%', '%메뉴2%', 4500],
        ['%안암%1호%', '%메뉴2%', 5000],
        ['%안암%2호%', '%메뉴1%', 4700],
        ['%안암%2호%', '%메뉴2%', 4900],
        ['%종암%1호%', '%메뉴4%', 5300],
        ['%종암%1호%', '%메뉴5%', 2300],
        ['%종암%1호%', '%메뉴1%', 900]]

cur.executemany('''
    INSERT INTO sells(fk1, fk2, price)
    VALUES((SELECT pk FROM supplier WHERE name LIKE ? LIMIT 0,1),
           (SELECT pk FROM part WHERE name LIKE ? LIMIT 0,1),
           ?)
    ''', data)

In [19]:
cur.execute('SELECT * FROM sells')
cur.fetchall()

[(1, 1, 4500),
 (1, 2, 4500),
 (1, 2, 5000),
 (2, 1, 4700),
 (2, 2, 4900),
 (3, 4, 5300),
 (3, 5, 2300),
 (3, 1, 900)]

In [20]:
# JOIN 연산
# INNER JOIN
cur.execute('''
    SELECT city.name, supplier.name FROM city
    INNER JOIN supplier
    ON city.pk = supplier.fk
    ''')
cur.fetchall()

[('성북구', '안암 1호점'), ('성북구', '안암 2호점'), ('성북구', '종암 1호점')]

In [21]:
# JOIN 연산
# LEFT JOIN
# city 가 기준
cur.execute('''
    SELECT city.name, supplier.name FROM city
    LEFT JOIN supplier
    ON city.pk = supplier.fk
    ''')
cur.fetchall()

[('성북구', '안암 1호점'),
 ('성북구', '안암 2호점'),
 ('성북구', '종암 1호점'),
 ('중구', None),
 ('강북구', None),
 ('어쩌구', None),
 ('저쩌구', None)]

In [22]:
cur.execute('SELECT * FROM city')
cur.fetchall()

[(1, '성북구'), (2, '중구'), (3, '강북구'), (4, '어쩌구'), (5, '저쩌구')]

In [23]:
cur.fetchall()
# fetch 는 위 select 문 커서를 수행하고 끄집어 냈고, 다음 작업을 위해 커서 위치가 바뀜. 그 상태에서 fetch(빈 작업 리스트에서) 했으니 당연히 비어 있는 것

[]

In [24]:
# JOIN 연산
# RIGHT JOIN
# supplier 가 기준
cur.execute('''
    SELECT city.name, supplier.name FROM city
    RIGHT JOIN supplier
    ON city.pk = supplier.fk
    ''')
cur.fetchall()

[('성북구', '안암 1호점'), ('성북구', '안암 2호점'), ('성북구', '종암 1호점')]

In [26]:
con.close()

------------------------------------------------------------------------------

In [129]:
# 다시 시작한다면 기존의 db를 불러오는 것부터!
import sqlite3

con = sqlite3.connect('example3.db')
cur = con.cursor()

In [130]:
# JOIN 연산
# INNER JOIN
cur.execute('''
    SELECT city.name, supplier.name FROM city
    INNER JOIN supplier
    ON city.pk = supplier.fk
    ''')
cur.fetchall()

[('성북구', '안암 1호점'), ('성북구', '안암 2호점'), ('성북구', '종암 1호점')]

In [35]:
# JOIN 연산
# LEFT JOIN
# city 가 기준
cur.execute('''
    SELECT city.name, supplier.name FROM city
    LEFT JOIN supplier
    ON city.pk = supplier.fk
    ''')
cur.fetchall()

[('성북구', '안암 1호점'),
 ('성북구', '안암 2호점'),
 ('성북구', '종암 1호점'),
 ('중구', None),
 ('강북구', None),
 ('어쩌구', None),
 ('저쩌구', None)]

In [133]:
# JOIN 연산
# 기준만 알고 있으면 LEFT, RIGHT JOIN 헷갈일 일 없음.
cur.execute('''
    SELECT city.name, supplier.name FROM supplier
    RIGHT JOIN city
    ON city.pk = supplier.fk
    ''')
cur.fetchall()

[('성북구', '안암 1호점'),
 ('성북구', '안암 2호점'),
 ('성북구', '종암 1호점'),
 ('중구', None),
 ('강북구', None),
 ('어쩌구', None),
 ('저쩌구', None)]

In [132]:
# JOIN 연산
# RIGHT JOIN
# supplier 가 기준
cur.execute('''
    SELECT city.name, supplier.name FROM city
    RIGHT JOIN supplier
    ON city.pk = supplier.fk
    ''')
cur.fetchall()

[('성북구', '안암 1호점'), ('성북구', '안암 2호점'), ('성북구', '종암 1호점')]

In [ ]:
# 기준만 알고 있다면 LEFT, RIGHT 완전히 같은 커리일 수 있음

In [37]:
# JOIN 연산
# OUTER JOIN
# supplier 가 기준
cur.execute('''
    SELECT city.name, supplier.name FROM city
    OUTER JOIN supplier
    ON city.pk = supplier.fk
    ''')
cur.fetchall()
# SQLite 는 OUTER JOIN 지원 x
# -> 따라서 에러가 뜸.

OperationalError: unknown join type: OUTER

In [134]:
con.close()
# DB 닫기

### 데이터베이스 설계 구조 짜보는 예제 (아이튠즈)

In [ ]:
# 겹치는 애들 안 겹치는 애들 보면서 엔티티 후보
# 속성, 개별 테이블
#! 어디가 겹치는 정보고, 어디가 누구에게 속하는 정보인지를 확인.

In [ ]:
# Track -> 겹치는 것 없음. 곡에 대한 정보. 별도의 엔티티로 동작함을 알 수 있음
# Length -> 곡에 붙어 있는 것. 곡에 하나의 attribute로 동작해야 할 것.
# Rating -> 곡에 걸려 있을 것. 하나의 attribute
# Count -> 몇 번 플레이 했는지도 attribute

In [39]:
# Track Relation 은 저 위의 4개로 이루어져 있을 것. 하나의 엔티티가 됨.
# 이 엔티티와 상관없이 중복적으로 값이 생길 수 있는 것은
# Artist, Album, Genre
# 트랙과 다른 엔티티

In [ ]:
# 앨범은 아티스트에 종속됨. 곡 -> 앨범 -> 가수
# 장르는 앨범보다는 곡에 붙는 것이 맞음
# 이를 바탕으로 엔티티를 뽑아냄.

In [ ]:
# 이번 예제에서는 뷰 테이블을 별도로 생성해볼 것(관계를 보여줌)
# Track - id(PK, 각각의 트랙을 구분할 수 있는), title, length, rating, count, album_id(FK), genre_id(FK)
# Album - id(PK), title, artist_id(FK)
# Genre - id(PK), name(CHAR - SQLite에서는 TEXT로 지원함)
# Artist - id(PK), name

In [3]:
con = sqlite3.connect('playlist4.db')
cur = con.cursor()

In [4]:
cur.executescript('''
    CREATE TABLE artist (
            pk INTEGER PRIMARY KEY,
            name TEXT DEFAULT '무명'
        );
    CREATE TABLE album (
            pk INTEGER PRIMARY KEY,
            name TEXT DEFAULT '무제',
            fk INTEGER NOT NULL
        );
    CREATE TABLE genre (
            pk INTEGER PRIMARY KEY,
            name TEXT DEFAULT '장르없음'
        );
    CREATE TABLE track (
            pk INTEGER PRIMARY KEY,
            name TEXT DEFAULT 'Track',
            length INTEGER DEFAULT 0,
            rating INTEGER DEFAULT 0,
            count INTEGER DEFAULT 0,
            fk1 INTEGER NOT NULL,
            fk2 INTEGER NOT NULL
        );
''')

In [5]:
cur.execute('INSERT INTO artist VALUES(?,?)', [None, '가수1'])
cur.execute('SELECT * FROM artist')
cur.fetchall()

[(1, '가수1')]

In [6]:
# named 스타일
cur.execute('INSERT INTO artist VALUES(:pk, :name)',
            {'pk':None, 'name':'가수2'})
cur.execute('SELECT * FROM artist')
cur.fetchall()

[(1, '가수1'), (2, '가수2')]

In [7]:
# executemany 가지고 pk 값 없이 입력
data = [['가수3'], ['가수4']]
cur.executemany('INSERT INTO artist (name) VALUES (?)', data)
# 왜 리스트로 한 번 더 감싸는지? -> executemany 이기 떄문
cur.execute('SELECT * FROM artist')
cur.fetchall()

[(1, '가수1'), (2, '가수2'), (3, '가수3'), (4, '가수4')]

In [8]:
cur.lastrowid
# 제일 마지막 

4

In [9]:
cur.executescript('''
    INSERT INTO genre(pk, name) VALUES (NULL, '장르1');
    INSERT INTO genre(name) VALUES ('장르2');
    INSERT INTO genre VALUES (NULL, '장르3');
    INSERT INTO genre VALUES (NULL, '장르4');
''')

In [10]:
cur.execute('SELECT * FROM genre')
cur.fetchall()

[(1, '장르1'), (2, '장르2'), (3, '장르3'), (4, '장르4')]

In [11]:
cur.execute('SELECT * FROM artist')
artist = cur.fetchall()
# artist 사용 가능하도록 만듦

In [12]:
cur.execute('SELECT * FROM artist WHERE name=?', ['가수1'])
cur.fetchall()

[(1, '가수1')]

In [13]:
cur.execute('SELECT * FROM artist WHERE name LIKE ?', ['%1'])
cur.fetchall()

[(1, '가수1')]

In [14]:
data = ['1', '2', '3', '4']
#? list 2개로 안 묶어줘도 되는건지?

for val in data:
    cur.execute('SELECT pk FROM artist WHERE name LIKE ?', ['%'+val])
    pk = cur.fetchall()
    if len(pk) > 0:
        # cur.execute('INSERT INTO album(pk, name, fk) VALUES (?, ?, ?)')
        # cur.execute('INSERT INTO album VALUES (NULL, ?, ?)')
        # cur.execute('INSERT INTO album(name, fk) VALUES (?, ?)')
        cur.execute('INSERT INTO album VALUES (NULL, ?, ?)',
                    ['앨범' + val, pk[0][0]])

In [15]:
pk  # 리스트로 묶여 있고 그 안에 튜플로 묶여있음. 따라서 위에서 [0][0] 으로 인덱싱 해준 것.

[(4,)]

In [16]:
cur.execute('SELECT * FROM album')
cur.fetchall()

[(1, '앨범1', 1), (2, '앨범2', 2), (3, '앨범3', 3), (4, '앨범4', 4)]

In [17]:
cur.execute('SELECT * FROM artist')
cur.fetchall()

[(1, '가수1'), (2, '가수2'), (3, '가수3'), (4, '가수4')]

In [18]:
data = [('싱글1', '%1'), ('싱글2', '%2'), ('싱글3', '%3'), ('싱글4', '%4')]

cur.executemany('''
    INSERT INTO album(name, fk) VALUES (?, (
        SELECT pk FROM artist WHERE name LIKE ?
    ))
''', data)
# '?' 의 개수대로 있어야 함

In [19]:
cur.execute('SELECT * FROM album')
cur.fetchall()

[(1, '앨범1', 1),
 (2, '앨범2', 2),
 (3, '앨범3', 3),
 (4, '앨범4', 4),
 (5, '싱글1', 1),
 (6, '싱글2', 2),
 (7, '싱글3', 3),
 (8, '싱글4', 4)]

In [20]:
cur.execute('SELECT * FROM album')
FK1 = cur.fetchall()

cur.execute('SELECT * FROM genre')
FK2 = cur.fetchall()

In [21]:
FK1

[(1, '앨범1', 1),
 (2, '앨범2', 2),
 (3, '앨범3', 3),
 (4, '앨범4', 4),
 (5, '싱글1', 1),
 (6, '싱글2', 2),
 (7, '싱글3', 3),
 (8, '싱글4', 4)]

In [22]:
for row in FK1:
    if row[1] == '싱글1':
        fk1 = row[0]
        break
fk1

5

In [23]:
for row in FK2:
    if row[1] == '장르3':
        fk2 = row[0]
        break
fk2

3

In [24]:
cur.execute('''
    INSERT INTO track (pk, name, length, rating, count, fk1, fk2)
    VALUES(NULL, ?, ?, ?, ?, ?, ?)
''', ['노래1', 270, 5, 100, fk1, fk2])

# map, filter 나중에 사용할 것

In [25]:
cur.execute('SELECT * FROM track')
cur.fetchall()

[(1, '노래1', 270, 5, 100, 5, 3)]

In [26]:
# 디폴트 활용
cur.execute('''
    INSERT INTO track (name, fk1, fk2)
    VALUES(?, ?, ?)
''', ['노래2', fk1, fk2])

# map, filter 나중에 사용할 것

In [27]:
cur.execute('SELECT * FROM track')
cur.fetchall()

[(1, '노래1', 270, 5, 100, 5, 3), (2, '노래2', 0, 0, 0, 5, 3)]

In [28]:
cur.execute('DELETE FROM track WHERE pk = 2')

In [29]:
cur.execute('SELECT * FROM track')
cur.fetchall()

[(1, '노래1', 270, 5, 100, 5, 3)]

In [30]:
cur.execute('INSERT INTO track(name, fk1, fk2) VALUES (?, ?, ?)',
            ['노래2', 1, 2])

In [31]:
cur.execute('SELECT * FROM track')
cur.fetchall()
# 만약 PK 설정 할 떄, AUTOINCREMENT 가 PK에 있으면 지금 추가한 것이 3이 될 것.

[(1, '노래1', 270, 5, 100, 5, 3), (2, '노래2', 0, 0, 0, 1, 2)]

In [32]:
cur.execute('''
    INSERT INTO track(name, fk1, fk2) VALUES (?, ?, (
        SELECT pk FROM genre WHERE name LIKE ?
    ))
    ''', ['노래3', 2, '%4'])

In [33]:
cur.execute('SELECT * FROM track')
cur.fetchall()

[(1, '노래1', 270, 5, 100, 5, 3),
 (2, '노래2', 0, 0, 0, 1, 2),
 (3, '노래3', 0, 0, 0, 2, 4)]

In [34]:
cur.execute('''
    INSERT INTO track(name, fk1, fk2) VALUES(?,(
        SELECT pk FROM album WHERE name LIKE ?
    ),(
        SELECT pk FROM genre WHERE name LIKE ?
    ))
''', ['노래4', '%싱글1%', '%장르3%'])

In [35]:
data = [
    ['노래5', '%싱글3%', '%장르4%'],
    ['노래6', '%앨범1%', '%장르1%'],
    ['노래7', '%앨범2%', '%장르2%'],
    ['노래8', '%앨범3%', '%장르3%'],
    ['노래9', '%앨범4%', '%장르4%'],
    ['노래10', '%싱글4%', '%장르3%'],
    ['노래11', '%싱글3%', '%장르2%'],
    ['노래12', '%싱글2%', '%장르1%'],
]
cur.executemany('''
    INSERT INTO track(name, fk1, fk2) VALUES(?,(
        SELECT pk FROM album WHERE name LIKE ?
    ),(
        SELECT pk FROM genre WHERE name LIKE ?
    ))
''', data)

In [36]:
cur.execute('SELECT * FROM track')
cur.fetchall()

[(1, '노래1', 270, 5, 100, 5, 3),
 (2, '노래2', 0, 0, 0, 1, 2),
 (3, '노래3', 0, 0, 0, 2, 4),
 (4, '노래4', 0, 0, 0, 5, 3),
 (5, '노래5', 0, 0, 0, 7, 4),
 (6, '노래6', 0, 0, 0, 1, 1),
 (7, '노래7', 0, 0, 0, 2, 2),
 (8, '노래8', 0, 0, 0, 3, 3),
 (9, '노래9', 0, 0, 0, 4, 4),
 (10, '노래10', 0, 0, 0, 8, 3),
 (11, '노래11', 0, 0, 0, 7, 2),
 (12, '노래12', 0, 0, 0, 6, 1)]

##### 어떤 테이블에서 어떤 값을 가져오는지 집중해서 입력하기!

### **집계함수** 등 사용

In [176]:
cur.execute('SELECT * FROM track')
cur.fetchall()

[(1, '노래1', 270, 5, 100, 5, 3),
 (2, '노래2', 0, 0, 0, 1, 2),
 (3, '노래3', 0, 0, 0, 2, 4),
 (4, '노래4', 0, 0, 0, 5, 3),
 (5, '노래5', 0, 0, 0, 7, 4),
 (6, '노래6', 0, 0, 0, 1, 1),
 (7, '노래7', 0, 0, 0, 2, 2),
 (8, '노래8', 0, 0, 0, 3, 3),
 (9, '노래9', 0, 0, 0, 4, 4),
 (10, '노래10', 0, 0, 0, 8, 3),
 (11, '노래11', 0, 0, 0, 7, 2),
 (12, '노래12', 0, 0, 0, 6, 1)]

In [175]:
# 집계함수 COUNT
cur.execute('SELECT COUNT(*) FROM track')
cur.fetchall()

[(12,)]

In [178]:
cur.execute('SELECT fk2, COUNT(*) FROM track GROUP BY fk2')
cur.fetchall()

[(1, 2), (2, 3), (3, 4), (4, 3)]

In [179]:
cur.execute('''
    SELECT T_B.pk, T_B.name, T_A.B name FROM genre AS T_B
    INNER JOIN
    (SELECT fk2 AS A, COUNT(*) AS B FROM track GROUP BY fk2) AS T_A
    ON T_A.A = T_B.pk
''')
cur.fetchall()

[(1, '장르1', 2), (2, '장르2', 3), (3, '장르3', 4), (4, '장르4', 3)]

In [180]:
cur.execute('''
    SELECT T_B.pk, T_B.name, T_A.B name FROM genre AS T_B
    INNER JOIN
    (SELECT fk2 AS A, COUNT(*) AS B FROM track GROUP BY fk2) AS T_A
    ON T_A.A = T_B.pk
    WHERE T_A.B > 2
    ORDER BY T_B.pk DESC
    LIMIT 0, 2
''')
cur.fetchall()

# LIMIT 는 상위 몇 개 필요한 지(DESC 이기 때문)

[(4, '장르4', 3), (3, '장르3', 4)]

In [ ]:
# 가수, 가수의 앨범 수 <- 출력하시오.

In [81]:
cur.execute('SELECT * FROM artist')
cur.fetchall()

[(1, '가수1'), (2, '가수2'), (3, '가수3'), (4, '가수4')]

In [82]:
cur.execute('SELECT * FROM album')
cur.fetchall()

[(1, '앨범1', 1),
 (2, '앨범2', 2),
 (3, '앨범3', 3),
 (4, '앨범4', 4),
 (5, '싱글1', 1),
 (6, '싱글2', 2),
 (7, '싱글3', 3),
 (8, '싱글4', 4)]

In [86]:
# 내가 작성해 본 Query
cur.execute('''
        SELECT T_B.name, T_A.B FROM artist AS T_B
        INNER JOIN
        (SELECT fk AS A, COUNT(*) AS B FROM album GROUP BY fk) AS T_A
        ON T_A.A = T_B.pk
''')
cur.fetchall()

[('가수1', 2), ('가수2', 2), ('가수3', 2), ('가수4', 2)]

In [87]:
# 류기곤 선생님 Query
cur.execute('''
    SELECT T_A.pk, T_A.name, T_B.CNT FROM artist AS T_A
    INNER JOIN
    (SELECT fk, COUNT(*) AS CNT FROM album GROUP BY fk) AS T_B
    ON T_B.fk = T_A.pk
    
''')
cur.fetchall()

[(1, '가수1', 2), (2, '가수2', 2), (3, '가수3', 2), (4, '가수4', 2)]

In [88]:
cur.execute('''
    SELECT track.pk, track.name, genre.name FROM track
    INNER JOIN genre ON genre.pk = track.fk2
''')
cur.fetchall()

[(1, '노래1', '장르3'),
 (2, '노래2', '장르2'),
 (3, '노래3', '장르4'),
 (4, '노래4', '장르3'),
 (5, '노래5', '장르4'),
 (6, '노래6', '장르1'),
 (7, '노래7', '장르2'),
 (8, '노래8', '장르3'),
 (9, '노래9', '장르4'),
 (10, '노래10', '장르3'),
 (11, '노래11', '장르2'),
 (12, '노래12', '장르1')]

In [89]:
# genre 별로 sorting
cur.execute('''
    SELECT track.pk, track.name, genre.name FROM track
    INNER JOIN genre ON genre.pk = track.fk2
    ORDER BY genre.name ASC
''')
cur.fetchall()

[(6, '노래6', '장르1'),
 (12, '노래12', '장르1'),
 (2, '노래2', '장르2'),
 (7, '노래7', '장르2'),
 (11, '노래11', '장르2'),
 (1, '노래1', '장르3'),
 (4, '노래4', '장르3'),
 (8, '노래8', '장르3'),
 (10, '노래10', '장르3'),
 (3, '노래3', '장르4'),
 (5, '노래5', '장르4'),
 (9, '노래9', '장르4')]

In [90]:
# 먼저 album 별로 sorting하고
# 겹치는 것이 있으면 genre 별로 sorting
cur.execute('''
    SELECT track.pk, track.name, album.name, genre.name FROM track
    INNER JOIN genre ON genre.pk = track.fk2
    INNER JOIN album ON album.pk = track.fk1
    ORDER BY album.name, genre.name
''')
cur.fetchall()

[(1, '노래1', '싱글1', '장르3'),
 (4, '노래4', '싱글1', '장르3'),
 (12, '노래12', '싱글2', '장르1'),
 (11, '노래11', '싱글3', '장르2'),
 (5, '노래5', '싱글3', '장르4'),
 (10, '노래10', '싱글4', '장르3'),
 (6, '노래6', '앨범1', '장르1'),
 (2, '노래2', '앨범1', '장르2'),
 (7, '노래7', '앨범2', '장르2'),
 (3, '노래3', '앨범2', '장르4'),
 (8, '노래8', '앨범3', '장르3'),
 (9, '노래9', '앨범4', '장르4')]

In [91]:
cur.execute('''
    SELECT track.pk, artist.name, track.name, album.name, genre.name FROM track
    INNER JOIN genre ON genre.pk = track.fk2
    INNER JOIN album ON album.pk = track.fk1
    INNER JOIN artist ON artist.pk = album.fk
    ORDER BY artist.name, album.name, genre.name
''')
cur.fetchall()

[(1, '가수1', '노래1', '싱글1', '장르3'),
 (4, '가수1', '노래4', '싱글1', '장르3'),
 (6, '가수1', '노래6', '앨범1', '장르1'),
 (2, '가수1', '노래2', '앨범1', '장르2'),
 (12, '가수2', '노래12', '싱글2', '장르1'),
 (7, '가수2', '노래7', '앨범2', '장르2'),
 (3, '가수2', '노래3', '앨범2', '장르4'),
 (11, '가수3', '노래11', '싱글3', '장르2'),
 (5, '가수3', '노래5', '싱글3', '장르4'),
 (8, '가수3', '노래8', '앨범3', '장르3'),
 (10, '가수4', '노래10', '싱글4', '장르3'),
 (9, '가수4', '노래9', '앨범4', '장르4')]

In [92]:
cur.execute('''
    SELECT track.pk, artist.name, track.name, album.name, genre.name FROM track
    INNER JOIN genre ON genre.pk = track.fk2
    INNER JOIN album ON album.pk = track.fk1
    INNER JOIN artist ON artist.pk = album.fk
    WHERE artist.name LIKE '%1'
    ORDER BY artist.name, album.name, genre.name
''')
cur.fetchall()

[(1, '가수1', '노래1', '싱글1', '장르3'),
 (4, '가수1', '노래4', '싱글1', '장르3'),
 (6, '가수1', '노래6', '앨범1', '장르1'),
 (2, '가수1', '노래2', '앨범1', '장르2')]

In [181]:
# WHERE 조건절이 없을 때.
cur.execute('''
    SELECT track.pk, artist.name, track.name, album.name, genre.name FROM track
    INNER JOIN genre ON genre.pk = track.fk2
    INNER JOIN album ON album.pk = track.fk1
    INNER JOIN artist ON artist.pk = album.fk
    ORDER BY artist.name, album.name, genre.name
''')
cur.fetchall()

[(1, '가수1', '노래1', '싱글1', '장르3'),
 (4, '가수1', '노래4', '싱글1', '장르3'),
 (6, '가수1', '노래6', '앨범1', '장르1'),
 (2, '가수1', '노래2', '앨범1', '장르2'),
 (12, '가수2', '노래12', '싱글2', '장르1'),
 (7, '가수2', '노래7', '앨범2', '장르2'),
 (3, '가수2', '노래3', '앨범2', '장르4'),
 (11, '가수3', '노래11', '싱글3', '장르2'),
 (5, '가수3', '노래5', '싱글3', '장르4'),
 (8, '가수3', '노래8', '앨범3', '장르3'),
 (10, '가수4', '노래10', '싱글4', '장르3'),
 (9, '가수4', '노래9', '앨범4', '장르4')]

In [93]:
# 위와 같은 구문 -> ON 절에 조건이 붙음.
cur.execute('''
    SELECT track.pk, artist.name, track.name, album.name, genre.name FROM track
    INNER JOIN genre ON genre.pk = track.fk2
    INNER JOIN album ON album.pk = track.fk1
    INNER JOIN artist ON artist.pk = album.fk and artist.name LIKE '%1'
    ORDER BY artist.name, album.name, genre.name
''')
cur.fetchall()

[(1, '가수1', '노래1', '싱글1', '장르3'),
 (4, '가수1', '노래4', '싱글1', '장르3'),
 (6, '가수1', '노래6', '앨범1', '장르1'),
 (2, '가수1', '노래2', '앨범1', '장르2')]

In [ ]:
cur.execute('''
    SELECT track.pk, artist.name, track.name, album.name, genre.name FROM track
    INNER JOIN genre ON genre.pk = track.fk2
    INNER JOIN album ON album.pk = track.fk1
    INNER JOIN artist ON artist.pk = album.fk
    WHERE track.count < 10
    ORDER BY artist.name, album.name, genre.name
''')
cur.fetchall()

[(1, '가수1', '노래1', '싱글1', '장르3'),
 (4, '가수1', '노래4', '싱글1', '장르3'),
 (6, '가수1', '노래6', '앨범1', '장르1'),
 (2, '가수1', '노래2', '앨범1', '장르2')]

In [ ]:
# 기억해야 할 것
#! 조인. 누굴 기준으로 해서 합칠 것이냐. INNER JOIN 부분집합을 어떻게 구할 것이냐.
#! 그렇게 부분집합 걸어넣고 내가 원하는 데이터를 얼만큼 가져올 것이냐 (WHERE 조건)
#! 정렬은 어떻게 할 것이냐.
#! 현재 어디에 속한 누가 얼마나 있는지 (집계함수)

## **ORM**

In [ ]:
# ORM 이란?
# Object Relational Mapping
# programming 테크닉. 즉, DB랑 상관없음.
# 그런데 데이터베이스와 비즈니스 로직이 서로 다르다는 점에서 필요성이 드러남.
# ORM은 OOP(객체지향)관점에서 데이터들을 오브젝트로 매핑해줌.

In [ ]:
# ERD -> 엔티티, 관계를 나타낸 다이어그램
# 이것만 보고는 테이블이 어떻게 이루어져 있는지는 모름.
# 2개, 3개(관계까지 테이블로 만들어서) 테이블이 될 수 있음. 방법은 여러가지. ERD만 보고 테이블이 어떻게 구성될 지 판단하기 어려움.
# 데이터 관점에서 봤을 떄 고객과 은행계좌는 다른 엔티티.

In [ ]:
# ORM 은 접근하는 방식이 다름. 객체지향 프로그램 처럼 프로퍼티 사용 등 특징이 있음.
# DB에 구현해놓은 것과 우리가 이용하는 방식이 다르기 떄문에, 코드도 복잡해지고 시간도 많이 걸리게 되는데,
# 그것을 줄여보자는 취지에서 ORM을 사용하게 됨.
# 즉 미스매치. 실제 RDB에 저장되어 있는 것들과 우리가 사용하는 것이 미스매치.
# ORM이란 기능/테크닉은 데이터베이스와 비즈니스 오브젝트 사이에 매핑을 해줌.
# 즉 이제 우리는 관계형 DB를 할 줄 알기 때문에 ORM 을 배울 것.

In [ ]:

#! ORM을 쓰는 근본적인 이유
# 프로그래머들이 데이터베이스를 어떻게 관리하고 이용하는지 보다는 로직 별로 프로그래밍 하는데 더 집중할 수 있도록 필요함
# 단점으로는 overhead가 있음.
# 내가 object를 건들면 ORM이 query로 처리해줌. 그걸 또 처리하는 한단계가 더 있어서 overhead가 존재함.

In [ ]:
# 모델과 실제 구현 결과물은 얼마든지 다를 수 있다!!!
# 사용자 태그 예시(인스타그램과 유사)

In [ ]:
1. 새 게시물 생성
2. 생성 시, 생성자는 사용자태그를 0 ~ N개 만들 수 있음
    -3. 사용자태그가 해시태그 풀에 있는지 확인
    -4. 확인이 됐으면, 새 게시물과 해시태그 관계 만들어주기
    -5. 해시태그 풀에 있는 빈도정보 +1
[여기까지 게시물 하나 작성하는 상황]

In [ ]:
<<정보>>

posting - pk, title, content, date

hashtag - pk, name, count

posting - hashtag : posting.pk, hashtag.pk

In [182]:
con.close # 기존 거 (플레이리스트 DB) 닫기

<function Connection.close()>

In [183]:
con = sqlite3.connect('sns2.db')
cur = con.cursor()

In [184]:
cur.execute('SELECT CURRENT_TIMESTAMP')
cur.fetchall()

[('2023-03-06 11:57:23',)]

In [185]:
cur.executescript('''
    DROP TABLE IF EXISTS posting;
    CREATE TABLE posting(
        pk INTEGER PRIMARY KEY,
        title TEXT,
        content TEXT,
        regdate DATE DEFAULT CURRENT_TIMESTAMP
    );
    DROP TABLE IF EXISTS hashtag;
    CREATE TABLE hashtag(
        pk INTEGER PRIMARY KEY,
        name TEXT,
        count INTEGER DEFAULT 0
    );
    DROP TABLE IF EXISTS poshas;
    CREATE TABLE poshas(
        fk1 INTEGER NOT NULL,
        fk2 INTEGER NOT NULL
    );
''')

In [186]:
# def addPosting(title, content, *hashtag) -> ORM을 위해 나중에(commit으로 잘 돌아가는지 확인 후) 감싸줄 것.

# 1단계 - 새 게시물 생성
cur.execute('INSERT INTO posting(title, content) VALUES(?, ?)',
            ['제목1', '내용1'])
pid = cur.lastrowid
pid

1

In [187]:
cur.execute('SELECT * FROM posting')
cur.fetchall()

[(1, '제목1', '내용1', '2023-03-06 12:01:32')]

In [191]:
hashtag = ['태그1', '태그2']
tagids = list()

# 2단계 - 사용자 태그 만들기
#   3단계 - 사용자 태그가 해시태그 풀에 있는지 확인
for tag in hashtag:
    cur.execute('SELECT pk FROM hashtag WHERE name=?', [tag])
    # print(cur.fetchone())   --> 없으면 None
    tid = cur.fetchone()
    if tid is not None:
        tagids.append(tid[0])

tagids

[1, 2]

In [192]:
tid

(2,)

In [193]:
#   4단계 - 확인이 됐으면, 새 게시물과 해시태그 관계 만들어주기
#   5단계 - 해시태그 풀에 있는 빈도정보 +1

for tid in tagids:  
    cur.execute('''
        INSERT INTO poshas VALUES(?,?)
    ''', [pid, tid])
    
    cur.execute('''
        UPDATE hashtag
        SET count = count + 1
        WHERE pk = ?                
    ''', [tid])

In [190]:
# 태그풀 생성
tags = [['태그1'], ['태그2'], ['태그3']]
cur.executemany('INSERT INTO hashtag(name) VALUES(?)', tags)

In [37]:
con.commit()
#! 여기까지 잘 돌아감을 확인(실제 내 DB에 반영이 되면)하면, 함수로 감싸주면 됨

In [ ]:
# 여기까지 생성. 이제는 수정

In [195]:
cur.execute('SELECT * FROM poshas')
cur.fetchall()

[(1, 1), (1, 2)]

In [196]:
cur.execute('SELECT * FROM posting')
cur.fetchall()

[(1, '제목1', '내용1', '2023-03-06 12:01:32')]

In [197]:
cur.execute('SELECT * FROM hashtag')
cur.fetchall()

[(1, '태그1', 1), (2, '태그2', 1), (3, '태그3', 0)]

In [194]:
cur.execute('''
    SELECT posting.title, posting.content, posting.regdate, hashtag.name FROM poshas
    INNER JOIN posting ON posting.pk = fk1
    INNER JOIN hashtag ON hashtag.pk = fk2
''')
cur.fetchall()

[('제목1', '내용1', '2023-03-06 12:01:32', '태그1'),
 ('제목1', '내용1', '2023-03-06 12:01:32', '태그2')]

In [116]:
pk

[(4,)]

In [199]:
pid

1

In [198]:
# 태그 정보만 필요하다면,
cur.execute('''
    SELECT hashtag.name
    FROM poshas
    INNER JOIN posting ON posting.pk = fk1
    INNER JOIN hashtag ON hashtag.pk = fk2
    WHERE fk1 = ?
''', [pid])
cur.fetchall()
# 1번 포스팅에 해당되는 태그들만 가져올 수 있음. 위에서 WHERE 절만 추가한 것.

[('태그1',), ('태그2',)]

In [ ]:
# 제목1, 내용1, 태그1, 태그2
# -> 제목1-1, 내용1, 태그1, 태그3
# 이렇게 바꿔보자!

In [200]:
cur.execute('UPDATE posting SET title=?, content=?',
            ['제목1-1', '내용1'])

In [201]:
cur.rowcount

1

In [202]:
hashtag = ['태그1', '태그3']
oldtag = list()
tagids = list()

cur.execute('SELECT fk2 FROM poshas WHERE fk1=?', [pid])
for row in cur.fetchall():
    oldtag.append(row[0])
    # 기존의 태그 목록
    
for tag in hashtag:
    cur.execute('SELECT pk FROM hashtag WHERE name=?', [tag])
    # print(cur.fetchone())   --> 없으면 None
    tid = cur.fetchone()
    if tid is not None:
        tagids.append(tid[0])

oldtag, tagids

([1, 2], [1, 3])

In [203]:
for tid in oldtag:
    if tid not in tagids:  # 없으면
        cur.execute('DELETE FROM poshas WHERE fk1=? AND fk2=?',
                    [pid, tid])
        
        cur.execute('UPDATE hashtag SET count = count - 1 WHERE pk=?',
                    [tid])
        
        

In [205]:
for tid in tagids:
    if tid not in oldtag:  # 없으면
        cur.execute('INSERT INTO poshas VALUES(?,?)',
                    [pid, tid])
        
        cur.execute('UPDATE hashtag SET count = count + 1 WHERE pk=?',
                    [tid])
        
        

In [206]:
cur.execute('SELECT * FROM poshas')
cur.fetchall()

[(1, 1), (1, 3), (1, 3)]

In [207]:
cur.execute('SELECT * FROM hashtag')
cur.fetchall()

[(1, '태그1', 1), (2, '태그2', 0), (3, '태그3', 2)]

In [208]:
con.close()